In [1]:
import requests
import numpy as np
import pandas as pd

In [21]:
NFL2022 = pd.read_csv('Full_NFL_Stats_2022.csv')
NFL2022.head()

,Unnamed: 0,teamName,displayName,description,abbreviation,value,perGameValue,rank
0,0,Falcons,Fumbles,The number of times a player/team has fumbled ...,FUM,21.0,1.0,13.0
1,1,Falcons,Fumbles Lost,The number of times a fumble is recovered by t...,LST,12.0,NaN,4.0
2,2,Falcons,Forced Fumbles,The total number of forced fumbles.,FF,8.0,NaN,62.0
3,3,Falcons,Fumbles Recovered,The number of fumbles recovered.,FR,7.0,NaN,23.0
4,4,Falcons,Fumbles Touchdowns,The number of times a fumbles is recovered and...,FTD,1.0,NaN,8.0


In [22]:
NFL2022_df = NFL2022.drop(columns='Unnamed: 0')

In [23]:
NFL2022_df.head()

,teamName,displayName,description,abbreviation,value,perGameValue,rank
0,Falcons,Fumbles,The number of times a player/team has fumbled ...,FUM,21.0,1.0,13.0
1,Falcons,Fumbles Lost,The number of times a fumble is recovered by t...,LST,12.0,NaN,4.0
2,Falcons,Forced Fumbles,The total number of forced fumbles.,FF,8.0,NaN,62.0
3,Falcons,Fumbles Recovered,The number of fumbles recovered.,FR,7.0,NaN,23.0
4,Falcons,Fumbles Touchdowns,The number of times a fumbles is recovered and...,FTD,1.0,NaN,8.0


In [33]:
NFL2022_df.sort_values(by=['rank'], ascending=False)

,teamName,displayName,description,abbreviation,value,perGameValue,rank
4983,Giants,Quarterback Rating,Quarterback Rating.,RAT,91.111664,92.513611,114.0
4964,Giants,Passer Rating,Passer Rating,RTG,91.111664,92.513611,114.0
7175,Buccaneers,Quarterback Rating,Quarterback Rating.,RAT,89.777500,85.675766,106.0
7156,Buccaneers,Passer Rating,Passer Rating,RTG,89.777500,85.675766,106.0
6353,Chargers,Quarterback Rating,Quarterback Rating.,RAT,93.334990,90.188423,102.0
...,...,...,...,...,...,...,...
6352,Chargers,Net Yards Per Pass Attempt,The average number net yards (taking sack yard...,NYDS/PA,6.112000,6.255814,NaN
7174,Buccaneers,Net Yards Per Pass Attempt,The average number net yards (taking sack yard...,NYDS/PA,5.932730,5.977778,NaN
7448,Commanders,Net Yards Per Pass Attempt,The average number net yards (taking sack yard...,NYDS/PA,5.767442,6.000000,NaN
7722,Panthers,Net Yards Per Pass Attempt,The average number net yards (taking sack yard...,NYDS/PA,6.077079,6.285714,NaN


In [25]:
NFL2022_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8768 entries, 0 to 8767
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   teamName      8768 non-null   object 
 1   displayName   8768 non-null   object 
 2   description   8768 non-null   object 
 3   abbreviation  8768 non-null   object 
 4   value         8768 non-null   float64
 5   perGameValue  3954 non-null   float64
 6   rank          8748 non-null   float64
dtypes: float64(3), object(4)
memory usage: 479.6+ KB


In [29]:
NFL2021 = pd.read_csv('Full_NFL_Stats_2021.csv')
NFL2021.head()

,Unnamed: 0,teamName,displayName,description,abbreviation,value,rank
0,0,Falcons,Fumbles,The number of times a player/team has fumbled ...,FUM,24.0,8.0
1,1,Falcons,Fumbles Lost,The number of times a fumble is recovered by t...,LST,11.0,4.0
2,2,Falcons,Forced Fumbles,The total number of forced fumbles.,FF,18.0,14.0
3,3,Falcons,Fumbles Recovered,The number of fumbles recovered.,FR,8.0,11.0
4,4,Falcons,Fumbles Touchdowns,The number of times a fumbles is recovered and...,FTD,0.0,12.0


In [30]:
NFL2021_df = NFL2021.drop(columns='Unnamed: 0')

In [32]:
NFL2021_df.sort_values(by=['rank'], ascending=False)

,teamName,displayName,description,abbreviation,value,rank
2791,Colts,Quarterback Rating,Quarterback Rating.,RAT,93.475840,102.0
2772,Colts,Passer Rating,Passer Rating,RTG,93.475840,102.0
4142,Vikings,Passer Rating,Passer Rating,RTG,101.629166,98.0
4161,Vikings,Quarterback Rating,Quarterback Rating.,RAT,101.629166,98.0
6079,Steelers,Quarterback Rating,Quarterback Rating.,RAT,85.272510,93.0
...,...,...,...,...,...,...
8753,Texans,Fumbles Lost,The number of times the ball has been fumbled ...,LST,7.000000,NaN
8762,Texans,Total Giveaways,The total number of giveaways.,TGV,22.000000,NaN
8765,Texans,Total Takeaways,The total number of takeaways.,TT,25.000000,NaN
8766,Texans,Total Drives,The total number of drives.,D,0.000000,NaN


In [34]:
NFL2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8768 entries, 0 to 8767
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    8768 non-null   int64  
 1   teamName      8768 non-null   object 
 2   displayName   8768 non-null   object 
 3   description   8768 non-null   object 
 4   abbreviation  8768 non-null   object 
 5   value         8768 non-null   float64
 6   rank          8668 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 479.6+ KB
